### import necessary libraries

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import geocoder

### obtain page as a string from the webpage

In [2]:
quote_page='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(quote_page).text

### extract table values and organize them into a dataframe and check

In [3]:
soup = BeautifulSoup(page, 'html.parser')
table = soup.find(lambda tag: tag.name=='table') 
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [value.text.strip('\n') for value in td]
    l.append(row)
l=l[1:]

df=pd.DataFrame(l, columns=["PostCode", "Borough", "Neighborhood"])

### drop rows that are not qualified and check

In [4]:
df.drop(df[(df['Borough']=='Not assigned') | (df['Borough']==None)].index,inplace=True)

### find rows whose Neighborhood value is "Not assigned" and print

In [5]:
neighborhood_not_assigned=df[df['Neighborhood']=='Not assigned']

### assign Borough cell value to Neighborhood cell if "Not assigned" and check

In [6]:
df.loc[neighborhood_not_assigned.index,'Neighborhood']=neighborhood_not_assigned['Borough']

### merge rows sharing the same PostCode

In [7]:
df = df.groupby('PostCode').agg({'Borough':'first', 
                             'Neighborhood': ', '.join}).reset_index()

### read in geospatial coordinates just in case

In [8]:
geospatial_coordinates=pd.read_csv('Geospatial_Coordinates.csv')

### define a function to get coordinates given postal code
#### first try to use the geocoder method, if tried after 3 times unsuccessfully then use read in geospatial coordinates instead

In [9]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    trial_count=0
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        trial_count+=1
        if trial_count>3:
            idx=geospatial_coordinates[geospatial_coordinates['Postal Code']==postal_code].index[0]
            lat_lng_coords=(geospatial_coordinates.iloc[idx]['Latitude'],geospatial_coordinates.iloc[idx]['Longitude'])

    return lat_lng_coords

In [10]:
col_lat=[]
col_lng=[]
count=0
for row in df.iterrows():
    postal_code=row[1]['PostCode']
    lat,lng=get_latlng(postal_code)
    col_lat.append(lat)
    col_lng.append(lng)
    print(count,postal_code)
    count+=1
df['Latitude']=col_lat
df['Longitude']=col_lng
df

0 M1B
1 M1C
2 M1E
3 M1G
4 M1H
5 M1J
6 M1K
7 M1L
8 M1M
9 M1N
10 M1P
11 M1R
12 M1S
13 M1T
14 M1V
15 M1W
16 M1X
17 M2H
18 M2J
19 M2K
20 M2L
21 M2M
22 M2N
23 M2P
24 M2R
25 M3A
26 M3B
27 M3C
28 M3H
29 M3J
30 M3K
31 M3L
32 M3M
33 M3N
34 M4A
35 M4B
36 M4C
37 M4E
38 M4G
39 M4H
40 M4J
41 M4K
42 M4L
43 M4M
44 M4N
45 M4P
46 M4R
47 M4S
48 M4T
49 M4V
50 M4W
51 M4X
52 M4Y
53 M5A
54 M5B
55 M5C
56 M5E
57 M5G
58 M5H
59 M5J
60 M5K
61 M5L
62 M5M
63 M5N
64 M5P
65 M5R
66 M5S
67 M5T
68 M5V
69 M5W
70 M5X
71 M6A
72 M6B
73 M6C
74 M6E
75 M6G
76 M6H
77 M6J
78 M6K
79 M6L
80 M6M
81 M6N
82 M6P
83 M6R
84 M6S
85 M7A
86 M7R
87 M7Y
88 M8V
89 M8W
90 M8X
91 M8Y
92 M8Z
93 M9A
94 M9B
95 M9C
96 M9L
97 M9M
98 M9N
99 M9P
100 M9R
101 M9V
102 M9W


,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
